In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import DataFrame 
import nltk
import random

from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression
from sklearn import neighbors
from scipy.spatial.distance import cosine
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

import re
import string
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_json("Office_Products_5.json", lines=True)
print(df.columns)
print(df.shape)

Index([u'asin', u'helpful', u'overall', u'reviewText', u'reviewTime',
       u'reviewerID', u'reviewerName', u'summary', u'unixReviewTime'],
      dtype='object')
(53258, 9)


In [3]:
df_required = df[['asin', 'reviewerID', 'overall']]
df_required

,asin,reviewerID,overall
0,B00000JBLH,A32T2H8150OJLU,5
1,B00000JBLH,A3MAFS04ZABRGO,5
2,B00000JBLH,A1F1A0QQP2XVH5,2
3,B00000JBLH,A49R5DBXXQDE5,5
4,B00000JBLH,A2XRMQA6PJ5ZJ8,5
5,B00000JBLH,A2JFOHC9W629IE,5
6,B00000JBLH,A38NELQT98S4H8,5
7,B00000JBLH,AA8M6331NI1EN,5
8,B00000JBLU,A25C2M3QF9G7OQ,5
9,B00000JBLU,A1RTVWTWZSIC94,5


In [4]:
#df_train = df_required.groupby('reviewerID', as_index=False).apply(lambda df: df.sample(int(len(df)*0.8)))
#df_train = pd.DataFrame(df_train).reset_index().drop(columns = ['level_0', 'level_1'])


In [5]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df_required, test_size=0.2, random_state=0, stratify=df_required[['asin', 'reviewerID', 'overall']])

In [15]:
import graphlab as gl
gl.product_key.set_product_key('B8FB-B1DD-C6F6-47E6-7984-0310-3DA5-4307')

In [19]:
sf_train = gl.SFrame({'user_id': train['reviewerID'],'item_id': train['asin'],'rating': train['overall']})
sf_test = gl.SFrame({'user_id': test['reviewerID'],'item_id': test['asin'],'rating': test['overall']})
sf_train

item_id,rating,user_id
B007B5VH0E,4,A260QLS8Z33KCQ
B00CODGOLK,5,A2E1EFNIZL2FVA
B00JAKXFQW,5,A1ZPY91VE3IDN1
B003VQR1T8,4,AS93JX4ZOK4WA
B0050BPWBQ,5,A1SBFI91YMPS9I
B009QZ49U2,5,A54REBNBBP8C0
B000LYAX1G,5,A25RYV8U6N0N1E
B005QAQFFS,4,A3MA0GENSE798M
B004VJSG2A,5,A81XIM8XFKWX5
B000GOZYRO,5,A1EE1HLYUIJPIG


In [119]:
model = gl.factorization_recommender.create(sf_train, target="rating")

Recsys training: model = factorization_recommender

Preparing data set.

Data has 42606 observations with 4905 users and 2420 items.

Data prepared in: 0.12912s

Training factorization_recommender for recommendations.

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 0.0127274

Final training RMSE: 0.11247

In [120]:
model.predict(sf_test)

dtype: float
Rows: 10652
[4.952044784527497, 4.738239190977292, 4.984453692537026, 4.5418739239389465, 2.1057482356841133, 4.252475141983704, 4.4745421739930675, 4.8429531226570175, 5.091253995400147, 4.297597866010146, 4.985629222791867, 4.874238043528275, 4.70360820877202, 3.7870022724086807, 4.145168868795262, 4.116479925316052, 3.8921819841193424, 4.166383251052575, 5.564881503086762, 2.010650600742774, 1.0073691899592445, 4.1237849886233375, 4.999169311922269, 4.866084441047387, 3.2314367214853332, 4.0624564881140754, 5.200639649730401, 4.9747244412715, 3.9006323809439705, 5.609193652605863, 4.660459070902543, 5.277727860462146, 4.215666884061174, 5.297743685048299, 3.6736876680219934, 4.654904916387276, 4.982435762386994, 4.164538621407227, 4.352399976428823, 5.098637721937375, 3.2868209476286934, 4.361424941074328, 5.152370184164719, 3.9369060466701553, 3.925257987778859, 4.996246933441834, 4.32980014334269, 5.054909742873387, 5.007713764410691, 2.9927102098877, 4.26665856639690

In [126]:
model.evaluate(sf_test, target = 'rating')

recommendations finished on 1000/3979 queries. users per second: 22765

recommendations finished on 2000/3979 queries. users per second: 27299.7

recommendations finished on 3000/3979 queries. users per second: 24273.8


Precision and recall summary statistics by cutoff
+--------+-------------------+-------------------+
| cutoff |   mean_precision  |    mean_recall    |
+--------+-------------------+-------------------+
|   1    |  0.00100527770797 | 0.000215715841501 |
|   2    |  0.00113093742146 | 0.000781184552233 |
|   3    | 0.000753958280975 | 0.000781184552233 |
|   4    | 0.000565468710731 | 0.000781184552233 |
|   5    | 0.000552902739382 | 0.000948730836894 |
|   6    | 0.000544525425149 |  0.00115816369272 |
|   7    | 0.000574444404552 |  0.00150821575174 |
|   8    | 0.000502638853983 |  0.00150821575174 |
|   9    | 0.000586411996314 |  0.00222028746155 |
|   10   | 0.000578034682081 |  0.00227683433263 |
+--------+-------------------+-------------------+
[10 rows x 3 columns]

('\nOverall RMSE: ', 0.8147026444750739)

Per User RMSE (best)
+---------------+-------+------------------+
|    user_id    | count |       rmse       |
+---------------+-------+------------------+
| ABNMD6670TV5

{'precision_recall_by_user': Columns:
 	user_id	str
 	cutoff	int
 	precision	float
 	recall	float
 	count	int
 
 Rows: 71622
 
 Data:
 +-----------------------+--------+-----------+--------+-------+
 |        user_id        | cutoff | precision | recall | count |
 +-----------------------+--------+-----------+--------+-------+
 | A00473363TJ8YSZ3YAGG9 |   1    |    0.0    |  0.0   |   2   |
 | A00473363TJ8YSZ3YAGG9 |   2    |    0.0    |  0.0   |   2   |
 | A00473363TJ8YSZ3YAGG9 |   3    |    0.0    |  0.0   |   2   |
 | A00473363TJ8YSZ3YAGG9 |   4    |    0.0    |  0.0   |   2   |
 | A00473363TJ8YSZ3YAGG9 |   5    |    0.0    |  0.0   |   2   |
 | A00473363TJ8YSZ3YAGG9 |   6    |    0.0    |  0.0   |   2   |
 | A00473363TJ8YSZ3YAGG9 |   7    |    0.0    |  0.0   |   2   |
 | A00473363TJ8YSZ3YAGG9 |   8    |    0.0    |  0.0   |   2   |
 | A00473363TJ8YSZ3YAGG9 |   9    |    0.0    |  0.0   |   2   |
 | A00473363TJ8YSZ3YAGG9 |   10   |    0.0    |  0.0   |   2   |
 +-------------------

In [127]:
model.recommend()

recommendations finished on 1000/4905 queries. users per second: 38254.1

recommendations finished on 2000/4905 queries. users per second: 33074.3

recommendations finished on 3000/4905 queries. users per second: 26543.5

recommendations finished on 4000/4905 queries. users per second: 26975.3

user_id,item_id,score,rank
A260QLS8Z33KCQ,B004FQRW9W,5.50641429375,1
A260QLS8Z33KCQ,B0019L0X38,5.45292246292,2
A260QLS8Z33KCQ,B00135HSI8,5.43785977314,3
A260QLS8Z33KCQ,B00006JNJR,5.42263996552,4
A260QLS8Z33KCQ,B0027JBLV4,5.38534998844,5
A260QLS8Z33KCQ,B001PMHURI,5.23251783798,6
A260QLS8Z33KCQ,B001V9LQH0,5.21542286823,7
A260QLS8Z33KCQ,B0000C3IDW,5.13263952683,8
A260QLS8Z33KCQ,B009ZMWHIC,5.11222362469,9
A260QLS8Z33KCQ,B00006IZSI,5.10990214298,10


In [128]:
model.evaluate_precision_recall(sf_test)

recommendations finished on 1000/3979 queries. users per second: 19042.5

recommendations finished on 2000/3979 queries. users per second: 23713.3

recommendations finished on 3000/3979 queries. users per second: 25639.3

{'precision_recall_by_user': Columns:
 	user_id	str
 	cutoff	int
 	precision	float
 	recall	float
 	count	int
 
 Rows: 71622
 
 Data:
 +-----------------------+--------+-----------+--------+-------+
 |        user_id        | cutoff | precision | recall | count |
 +-----------------------+--------+-----------+--------+-------+
 | A00473363TJ8YSZ3YAGG9 |   1    |    0.0    |  0.0   |   2   |
 | A00473363TJ8YSZ3YAGG9 |   2    |    0.0    |  0.0   |   2   |
 | A00473363TJ8YSZ3YAGG9 |   3    |    0.0    |  0.0   |   2   |
 | A00473363TJ8YSZ3YAGG9 |   4    |    0.0    |  0.0   |   2   |
 | A00473363TJ8YSZ3YAGG9 |   5    |    0.0    |  0.0   |   2   |
 | A00473363TJ8YSZ3YAGG9 |   6    |    0.0    |  0.0   |   2   |
 | A00473363TJ8YSZ3YAGG9 |   7    |    0.0    |  0.0   |   2   |
 | A00473363TJ8YSZ3YAGG9 |   8    |    0.0    |  0.0   |   2   |
 | A00473363TJ8YSZ3YAGG9 |   9    |    0.0    |  0.0   |   2   |
 | A00473363TJ8YSZ3YAGG9 |   10   |    0.0    |  0.0   |   2   |
 +-------------------

In [129]:
model.summary()

Class                            : FactorizationRecommender

Schema
------
User ID                          : user_id
Item ID                          : item_id
Target                           : rating
Additional observation features  : 0
User side features               : []
Item side features               : []

Statistics
----------
Number of observations           : 42606
Number of users                  : 4905
Number of items                  : 2420

Training summary
----------------
Training time                    : 4.9838

Model Parameters
----------------
Model class                      : FactorizationRecommender
num_factors                      : 8
binary_target                    : 0
side_data_factorization          : 1
solver                           : auto
nmf                              : 0
max_iterations                   : 50

Regularization Settings
-----------------------
regularization                   : 0.0
regularization_type              : normal
linear_regul